In [ ]:
from AnalyticalLabware import SpinsolveNMR
from chempiler import Chempiler
import ChemputerAPI
from utils import DiscoveryPlatform

In [2]:
from xdl import initialize_logging
initialize_logging(file_level=20, file_dir="log_files", stream_level=40)

In [ ]:
graph_file = 'minimal_monitoring_graph.json'

In [4]:
c = Chempiler(
    experiment_code="manual",
    graph_file=graph_file,
    output_dir='.',
    simulation=False,
    device_modules=[ChemputerAPI]
)

In [ ]:
nmr: SpinsolveNMR = c['nmr']
dp = DiscoveryPlatform(nmr=nmr, c=c)

In [ ]:
#nmr.protocols_list

In [ ]:
#nmr.cmd.get_protocol("1D WET SUP")

Define experimental parameters here

In [ ]:
# premix the contents of the flowcell and the reactor vial
from concurrent.futures import ThreadPoolExecutor, wait

executor = ThreadPoolExecutor(max_workers=3)

cycles = 30
pumps = ['p33', 'p32', 'p31']
pump_switch = True

dp.take_sample(pumps[0])

counter = 0

while counter < cycles or not pump_switch:
    futures = []
    if pump_switch:
        futures.append(executor.submit(dp.flowcell, pump1=pumps[0], pump2=pumps[1]))
        futures.append(executor.submit(dp.take_sample, pump=pumps[2]))
    else:
        futures.append(executor.submit(dp.flowcell, pump1=pumps[2], pump2=pumps[0]))
        futures.append(executor.submit(dp.return_sample, pump=pumps[1]))
    
    pump_switch = not pump_switch

    wait(futures)

    counter += 1
    print(counter, "/", cycles)

dp.return_sample(pumps[0])

In [ ]:
dp.test_nmr()

Define experimental parameters here

In [ ]:
nmr.sample = 'MGS259'
total_runtime = 5*60*60 # seconds

In [ ]:
from concurrent.futures import ThreadPoolExecutor, wait
from datetime import datetime, timedelta

executor = ThreadPoolExecutor(max_workers=3)

pumps = ['p33', 'p32', 'p31']

time_elapsed = 0
pump_switch = True

start_time = datetime.now()
last_shim_time = datetime.now() - timedelta(hours=1)
print(f"Starting sampling at {start_time}")

dp.take_sample(pumps[0])

counter = 0

while time_elapsed < total_runtime or not pump_switch:
    futures = []
    if pump_switch:
        futures.append(executor.submit(dp.flowcell, pump1=pumps[0], pump2=pumps[1]))
        futures.append(executor.submit(dp.take_sample, pump=pumps[2]))
    else:
        futures.append(executor.submit(dp.flowcell, pump1=pumps[2], pump2=pumps[0]))
        futures.append(executor.submit(dp.return_sample, pump=pumps[1]))
    
    pump_switch = not pump_switch

    wait(futures)

    if counter%5 == 0 and counter >= 10:
        time_since_last_shim = (datetime.now() - last_shim_time).total_seconds()
        print(f"Time since last shim: {str(timedelta(seconds=time_since_last_shim))}")
        if time_since_last_shim > 5 * 60:
            print("Shim too old. Recalibrating...")
            og_sample = nmr.sample
            nmr.sample = 'shim'
            dp.shim_on_sample(1.6)
            last_shim_time = datetime.now()
            nmr.sample = og_sample
        dp.get_wet_sup()

    time_elapsed = (datetime.now() - start_time).total_seconds()
    time_left = timedelta(seconds=total_runtime - time_elapsed)
    print(f"Time left: {str(time_left if time_left.total_seconds() > 0 else 0)}")

    counter += 1

print("Experiment finished.")
print("Returning sample")
dp.return_sample(pumps[0])

In [ ]:
# Recycle the sample in the flow cell if needed for yield determination

print("Purging flowcell")
dp.purge_flowcell('sample')

In [ ]:
dp.clean_flowcell('acetone', 3)

In [ ]:
dp.purge_flowcell()